In [1]:
import pandas as pd


In [19]:
file = '/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/dec24.txt'
out='/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/criteria_above_discard_redundant_clinsig.txt'

In [20]:
with open(file,'r') as f:
    with open(out,'w') as f_out:
        line=f.readline()
        f_out.write('clinvar_id;review_status;clinical_sig;variant_type;hgvs_p;missense\n')
        while True:
            line=f.readline()

            if line.count(';')>6:
                discard_num=line.count(';')-6+3
                line_new=';'.join(line.split(';')[:3]+line.split(';')[discard_num:])
                print(set(line.split(';'))-set(line_new.split(';')),' is discarded')
                f_out.write(line_new)
            else:
                f_out.write(line)
            if not line:break

In [21]:
data=pd.read_csv(out,delimiter=';')
data.head()

,clinvar_id,review_status,clinical_sig,variant_type,hgvs_p,missense
0,SCV000787438,"criteria provided, single submitter",Likely pathogenic,Variation,NP_004306.3:p.Tyr52Cys,missense variant
1,SCV002232601,"criteria provided, single submitter",Pathogenic,Variation,NP_001309903.1:p.Arg71Lys,missense variant
2,SCV002291827,"criteria provided, single submitter",Uncertain significance,Variation,NP_001309899.1:p.Gln90Glu,missense variant
3,SCV001445903,"criteria provided, single submitter",Likely pathogenic,Variation,NP_000276.2:p.Gly278Asp,missense variant
4,SCV001449002,"criteria provided, single submitter",Likely pathogenic,Variation,NP_000276.2:p.Gly448Arg,missense variant


In [22]:
data.clinical_sig.unique()

array(['Likely pathogenic', 'Pathogenic', 'Uncertain significance',
       'Benign', 'Likely benign', 'uncertain', 'likely pathogenic',
       'unknown', 'pathogenic', 'drug response', 'Uncertain Significance',
       'Likely Benign', 'other', 'risk factor', 'Affects',
       'Likely risk allele', 'MUT', 'benign', 'Uncertain risk allele',
       'no known pathogenicity', 'Established risk allele',
       'Pathogenic, low penetrance', 'association'], dtype=object)

In [26]:
data.variant_type.unique()

array(['Variation', 'single nucleotide variant', 'Indel',
       'copy number gain', 'Insertion'], dtype=object)

In [32]:
data[data.variant_type=='Indel']

,clinvar_id,review_status,clinical_sig,variant_type,hgvs_p,missense
697,SCV000114620,"criteria provided, single submitter",Uncertain significance,Indel,NP_733842.2:p.Trp2749Ser,missense variant
1962,SCV000227177,"criteria provided, single submitter",Uncertain significance,Indel,NP_001012740.1:p.Arg79Met,missense variant
2245,SCV000231070,"criteria provided, single submitter",Uncertain significance,Indel,NP_001163887.1:p.Tyr280Ile,missense variant
3564,SCV000250689,"criteria provided, single submitter",Likely pathogenic,Indel,NP_003027.1:p.Leu167Gly,missense variant
6883,SCV000337663,"criteria provided, single submitter",Uncertain significance,Indel,NP_490647.1:p.Lys320Ser,missense variant
...,...,...,...,...,...,...
634002,SCV001990765,"criteria provided, single submitter",Uncertain significance,Indel,NP_000247.2:p.Ala848Trp,missense variant
634041,SCV001990282,"criteria provided, single submitter",Uncertain significance,Indel,NP_001338763.1:p.Arg1466Pro,missense variant
634184,SCV001999140,"criteria provided, single submitter",Uncertain significance,Indel,NP_002476.2:p.Gln451Pro,missense variant
635042,SCV002012747,"criteria provided, single submitter",Uncertain significance,Indel,NP_000170.1:p.Pro92Leu,missense variant


In [25]:
data[data.variant_type=='Duplication']

,clinvar_id,review_status,clinical_sig,variant_type,hgvs_p,missense


In [30]:
data[data.variant_type=='Complex']

,clinvar_id,review_status,clinical_sig,uniprot_kb,variant_type,hgvs_p,missense
1361,RCV000207037,"criteria provided, single submitter",Uncertain significance,O00754#VAR_068061,Complex,O00754:p.Gly800Trp,synonymous variant
2812,RCV001844011,"criteria provided, single submitter",Pathogenic,P10828#VAR_004634,Complex,P10828:p.Arg316His,missense variant


In [33]:
missense_data=data[data.missense=='missense variant']

# Process clinical significance

what is protective?

In [35]:
def if_positive_or_negative(string):
    if string in ['Pathogenic', 
                'Pathogenic/Likely pathogenic', 
                'probable-pathogenic',
                'Likely pathogenic', 
                'pathologic', 
                'pathogenic',
                'likely pathogenic',
                'Pathogenic/Likely pathogenic/Established risk allele',
                'likely pathogenic - adrenal pheochromocytoma',
                'Pathogenic/Pathogenic, low penetrance']:
        return 1
    elif string in ['Benign',
                'Likely benign',
                'Benign/Likely benign',
                'non-pathogenic', 
                'benign', 'probable-non-pathogenic', 'Likely Benign', 'probably not pathogenic',
    ]:
        return -1

    else: return 0


In [ ]:
missense_data['label']='1'

In [12]:
def generate_example(start,end):
    with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/ClinVarFullRelease_00-latest.xml', 'r') as f_in:
        with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/example_'+str(start)+'_'+str(end)+'.xml','w') as f_out:
            line_count=0
            while True:
                line=f_in.readline()
                line_count+=1
                if line_count>start and line_count<end:
                    f_out.write(line)
                if line_count>end:
                    break

def find_line(content):
    with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/ClinVarFullRelease_00-latest.xml', 'r') as f_in:
            line_count=0

            while True:
                line=f_in.readline()
                line_count+=1

                if content in line:
                    print(line_count)
                    break

In [30]:
generate_example(554695631,554696331)

In [28]:
find_line('SCV002515940')

554696231


In [ ]:
RCV000203850 L936 (932-1335)
SCV000259045 L1117

delayed one

SCV000258941 L1257